# Piaui Herald - Exploratory Data Analysis
Finding interesting cases for Rosie's column

In [1]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('../../../serenata-data/2017-03-15-reimbursements.xz',
                      dtype={'applicant_id': np.str,
                             'cnpj_cpf': np.str,
                             'congressperson_id': np.str,
                             'subquota_number': np.str,
                             'document_id': np.int},
                      low_memory=False)

## Luxury Hotel

We are aiming to find suspicious expenses in hotels, maybe someone spent the holidays in some luxury hotel and asked for reimbursement

In [2]:
lodging = dataset[dataset['subquota_description'] == 'Lodging, except for congressperson from Distrito Federal']
keys = ['congressperson_id','cnpj_cpf', 'supplier']
grouped = lodging.groupby(keys)

Number of Lodging expenses

In [3]:
len(grouped)

21063

In [4]:
subquota_numbers = grouped['subquota_number'].agg(lambda x: ','.join(x)).reset_index()
subquota_numbers.head()

,congressperson_id,cnpj_cpf,supplier,subquota_number
0,101309,02697172000191,HOTEL E RESTAURANTE CARAIVA,"14,14"
1,101309,04652379000175,HOTEL ROYAL LTDA,"14,14,14,14,14"
2,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,"14,14"
3,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,14
4,101309,07881411000182,ITEM HOTELARIA TURISMO ESTACIONAMENTO P/ VEICU...,14


In [5]:
document_ids = grouped['document_id'].agg(lambda x: tuple(x)).reset_index()
document_ids.head()

,congressperson_id,cnpj_cpf,supplier,document_id
0,101309,02697172000191,HOTEL E RESTAURANTE CARAIVA,"(5769892, 5769895)"
1,101309,04652379000175,HOTEL ROYAL LTDA,"(5789888, 5942187, 5978852, 6005283, 6019174)"
2,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,"(5796458, 5922475)"
3,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,"(6041598,)"
4,101309,07881411000182,ITEM HOTELARIA TURISMO ESTACIONAMENTO P/ VEICU...,"(5622880,)"


In [6]:
net_values_sum = grouped['total_net_value'].agg({'sum': np.sum}).reset_index()
net_values_sum.head()

,congressperson_id,cnpj_cpf,supplier,sum
0,101309,02697172000191,HOTEL E RESTAURANTE CARAIVA,128.5
1,101309,04652379000175,HOTEL ROYAL LTDA,1890.9
2,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,617.7
3,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,953.4
4,101309,07881411000182,ITEM HOTELARIA TURISMO ESTACIONAMENTO P/ VEICU...,194.0


In [7]:
aggregation = pd.merge(pd.merge(subquota_numbers, document_ids, on=keys),
                       net_values_sum, on=keys)
aggregation.head()

,congressperson_id,cnpj_cpf,supplier,subquota_number,document_id,sum
0,101309,02697172000191,HOTEL E RESTAURANTE CARAIVA,"14,14","(5769892, 5769895)",128.5
1,101309,04652379000175,HOTEL ROYAL LTDA,"14,14,14,14,14","(5789888, 5942187, 5978852, 6005283, 6019174)",1890.9
2,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,"14,14","(5796458, 5922475)",617.7
3,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,14,"(6041598,)",953.4
4,101309,07881411000182,ITEM HOTELARIA TURISMO ESTACIONAMENTO P/ VEICU...,14,"(5622880,)",194.0


Get net value by row

In [8]:
def get_top_net_value(row):
    l = list(row['document_id'])
    values = []
    for reimbursement_id in l:
        values.append(float(dataset[dataset['document_id'] == reimbursement_id]['total_net_value']))
    return {'top_net_value':max(values), 'top_document':l[values.index(max(values))]}

In [9]:
top_things = aggregation.apply(func=get_top_net_value, axis='columns')
# new columns
aggregation['top_net_value'], aggregation['top_document'] = "",""

In [10]:
for _ in range(len(top_things)):
    # paliative since DataFrame.replace() did not work ¯\_(ツ)_/¯
    aggregation.loc[_, 'top_net_value'] = top_things[_]['top_net_value']
    aggregation.loc[_, 'top_document'] = top_things[_]['top_document']

In [11]:
aggregation.head()

,congressperson_id,cnpj_cpf,supplier,subquota_number,document_id,sum,top_net_value,top_document
0,101309,02697172000191,HOTEL E RESTAURANTE CARAIVA,"14,14","(5769892, 5769895)",128.5,79,5769892
1,101309,04652379000175,HOTEL ROYAL LTDA,"14,14,14,14,14","(5789888, 5942187, 5978852, 6005283, 6019174)",1890.9,822.7,5978852
2,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,"14,14","(5796458, 5922475)",617.7,430.7,5796458
3,101309,07881411000182,ITEM HOTELARIA TURISMO E ESTACIONAMENTO PARA V...,14,"(6041598,)",953.4,953.4,6041598
4,101309,07881411000182,ITEM HOTELARIA TURISMO ESTACIONAMENTO P/ VEICU...,14,"(5622880,)",194.0,194,5622880


In [12]:
aggregation = aggregation.sort_values(by='top_net_value', ascending=False)
aggregation.head(10)

,congressperson_id,cnpj_cpf,supplier,subquota_number,document_id,sum,top_net_value,top_document
9102,160633,07959819000120,KAPITAL VIAG. E TUR. LTDA,"14,14,14,14,14,14","(5877120, 5952491, 5952505, 5988139, 6025233, ...",24842.08,11090.1,5877120
17690,74319,04443021000213,EUROPA INVESTIMENTOS LTDA,"14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,1...","(5037717, 5054130, 5054141, 5054148, 5072351, ...",139915.00,10665,5322461
8366,160606,05668014000100,D.A.N HOTEL SP LTDA,"14,14,14,14,14,14,14,14,14,14,14,14,14,14,14","(2301718, 2385930, 2414282, 2443857, 5058637, ...",53664.07,10170,5652142
6509,160549,40848905000130,WM TURISMO E VIAGENS LTDA.,14,"(2349567,)",9316.20,9316.2,2349567
13162,73437,58133323000133,VASCO DA GAMA TURISMO LTDA,"14,14","(5168791, 5487203)",16000.00,9000,5487203
12678,178981,61223608000170,MEGTUR AGENCIA DE VIAGENS E TURISMO LTDA,14,"(5794903,)",8510.00,8510,5794903
17693,74319,04863492000108,RIO POTY HOTEL,"14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,1...","(1643367, 1643368, 1839328, 1839336, 1867172, ...",61400.00,8050,1955426
11387,178847,16978175000108,Adria viagens e turismo LTDA ME,"14,14,14,14,14,14,14,14,14,14,14,14","(5677346, 5677362, 5781905, 5781908, 5803661, ...",34425.02,7904.84,5884760
17250,74207,04433548000186,EMILIANO EMPREENDIMENTOS E PARTICIPAÇÕES HOTEL...,"14,14,14,14,14,14","(5150333, 5237164, 5346066, 5443587, 5531077, ...",32982.39,7697.22,5443587
17121,74177,34776559000120,NOVO HOTEL,"14,14,14,14,14,14,14,14,14,14","(5207833, 5349869, 5366314, 5410698, 5467753, ...",32144.00,7600,5497082


## Top eaters

Who were the congresspeople that ate more in one day and when?

In [9]:
meals = dataset[dataset['subquota_description'] == 'Congressperson meal']

In [7]:
keys = ['congressperson_name', 'issue_date']
meals_aggregation = meals.groupby(keys)['total_net_value']. \
                        agg({'sum': np.sum, 'expenses': len, 'mean': np.mean})
meals_aggregation['expenses'] = meals_aggregation['expenses'].astype(np.int)

In [8]:
meals_aggregation.sort_values(['expenses', 'sum'], ascending=[False, False]).head(20)

sum  expenses       mean
congressperson_name issue_date                                        
CELSO MALDANER      2011-09-05 00:00:00.0  750.28        13  57.713846
JOSÉ PAULO TÓFFANO  2010-04-27 00:00:00.0  558.07        13  42.928462
SANDRA ROSADO       2012-01-12 00:00:00.0  333.40        12  27.783333
                    2012-01-17 00:00:00.0  287.43        12  23.952500
                    2012-01-06 00:00:00.0  281.75        12  23.479167
LÉO VIVAS           2010-08-31 00:00:00.0  630.00        11  57.272727
SANDRA ROSADO       2012-01-11 00:00:00.0  541.56        11  49.232727
PAULO WAGNER        2011-07-21 00:00:00.0  537.66        11  48.878182
SANDRA ROSADO       2015-01-07 00:00:00.0  396.60        11  36.054545
                    2012-01-15 00:00:00.0  295.58        11  26.870909
FRANCISCO DE ASSIS  2014-11-18 00:00:00.0  565.99        10  56.599000
SANDRA ROSADO       2012-01-09 00:00:00.0  340.18        10  34.018000
MAGDA MOFATTO       2015-03-15 00:00:00.0  435.63         9  48.403333
FRANCISCO DE ASSIS  2014-12-02 00:00:00.0  409.56         9  45.506667
PAULO WAGNER        2011-07-31 00:00:00.0  404.75         9  44.972222
FRANCISCO DE ASSIS  2014-10-01 00:00:00.0  346.19         9  38.465556
VANDERLEI MACRIS    2009-09-12 00:00:00.0  279.13         9  31.014444
                    2009-12-11 00:00:00.0  274.00         9  30.444444
SANDRA ROSADO       2012-01-05 00:00:00.0  265.87         9  29.541111
DR. ADILSON SOARES  2014-03-19 00:00:00.0  248.59         9  27.621111

What was the highest meal reimbursement, where it was made and by who?

In [16]:
meals[['document_id', 'issue_date', 'total_net_value', 'congressperson_name', 'supplier']]. \
        sort_values('total_net_value', ascending=False).head(10)

,document_id,issue_date,total_net_value,congressperson_name,supplier
1266040,5857053,2015-11-25 00:00:00.0,6205.00,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
1242665,5588112,2015-01-18 00:00:00.0,5880.00,MARLLOS SAMPAIO,VR COMERCIO DE VEICULOS LTDA-ME
23296,1689682,2009-09-01 00:00:00.0,5852.00,MANOEL SALVIANO,HOTEL NACIONAL S/A
767478,2383218,2012-08-22 00:00:00.0,5696.00,LIDERANÇA DO PT,PARRILLA FIGUEIRA DA VILLA
1467845,5954206,2016-03-30T00:00:00,5142.25,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
1266024,5834088,2015-11-03 00:00:00.0,5054.99,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
1126704,5573885,2014-12-10 00:00:00.0,5050.00,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
954390,5056362,2013-03-18 00:00:00.0,4990.00,LIDERANÇA DO PSDB,Emporio Albamonte Comercio e Serviços Alimentí...
23314,1717750,2009-11-03 00:00:00.0,4840.00,MANOEL SALVIANO,HOTEL NACIONAL S/A
389390,2195568,2011-11-02 00:00:00.0,4510.00,SALVADOR ZIMBALDI,HOTEL NACIONAL SA
